## Python Visualization

### Data Analysis

In [6]:
# Imports
import pandas as pd
import numpy as np
import panel as pn
pn.extension('tabulator')

import hvplot.pandas

In [195]:
# Import data
data=pd.read_csv("Data\\LasVegasTripAdvisorReviews-Dataset.csv")
data.info()

data["Hotel_name"].nunique() # 21 hotels.
data["Review_month"].nunique()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 504 entries, 0 to 503
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   User_country      504 non-null    object
 1   Nr_reviews        504 non-null    int64 
 2   Nr_hotel_reviews  504 non-null    int64 
 3   Helpful_votes     504 non-null    int64 
 4   Score             504 non-null    int64 
 5   Period_of_stay    504 non-null    object
 6   Traveler_type     504 non-null    object
 7   Pool              504 non-null    object
 8   Gym               504 non-null    object
 9   Tennis_court      504 non-null    object
 10  Spa               504 non-null    object
 11  Casino            504 non-null    object
 12  Free_internet     504 non-null    object
 13  Hotel_name        504 non-null    object
 14  Hotel_stars       504 non-null    int64 
 15  Nr_rooms          504 non-null    int64 
 16  User_continent    504 non-null    object
 17  Member_years    

12

In [200]:
data.head()

,User_country,Nr_reviews,Nr_hotel_reviews,Helpful_votes,Score,Period_of_stay,Traveler_type,Pool,Gym,Tennis_court,Spa,Casino,Free_internet,Hotel_name,Hotel_stars,Nr_rooms,User_continent,Member_years,Review_month,Review_weekday
0,USA,11,4,13,5,Dec-Feb,Friends,NO,YES,NO,NO,YES,YES,Circus Circus Hotel & Casino Las Vegas,3,3773,North America,9,January,Thursday
1,USA,119,21,75,3,Dec-Feb,Business,NO,YES,NO,NO,YES,YES,Circus Circus Hotel & Casino Las Vegas,3,3773,North America,3,January,Friday
2,USA,36,9,25,5,Mar-May,Families,NO,YES,NO,NO,YES,YES,Circus Circus Hotel & Casino Las Vegas,3,3773,North America,2,February,Saturday
3,UK,14,7,14,4,Mar-May,Friends,NO,YES,NO,NO,YES,YES,Circus Circus Hotel & Casino Las Vegas,3,3773,Europe,6,February,Friday
4,Canada,5,5,2,4,Mar-May,Solo,NO,YES,NO,NO,YES,YES,Circus Circus Hotel & Casino Las Vegas,3,3773,North America,7,March,Tuesday


In [206]:
# # Converting month column
# import calendar
# print({month: index for index, month in enumerate(calendar.month_abbr) if month})

# from datetime import datetime
# name = 'March'
# num = datetime.strptime(name, '%B').month
# print(num, type(num))

# month_list=[]
# for i in data['Review_month']:
#     name=i
#     num = datetime.strptime(name, '%B').month
#     month_list.append(num)
    
# month_list
# data['Review_month']=month_list
data['Hotel_stars'].unique()

array([3, 4, 5], dtype=int64)

In [260]:
# list = [102, 232, 424]
# count = 0
# d = {} #Empty dictionary to add values into

# for i in data['Hotel_name']:
#     d[count] = i
#     count+=1
    
list1 = ['Circus Circus Hotel & Casino Las Vegas',
       'Excalibur Hotel & Casino', 'Monte Carlo Resort&Casino',
       'Treasure Island- TI Hotel & Casino',
       'Tropicana Las Vegas - A Double Tree by Hilton Hotel',
       'Caesars Palace', 'The Cosmopolitan Las Vegas',
       'The Palazzo Resort Hotel Casino', 'Wynn Las Vegas',
       'Trump International Hotel Las Vegas', 'The Cromwell',
       'Encore at wynn Las Vegas',
       'Hilton Grand Vacations on the Boulevard',
       "Marriott's Grand Chateau", 'Tuscany Las Vegas Suites & Casino',
       'Hilton Grand Vacations at the Flamingo', 'Wyndham Grand Desert',
       'The Venetian Las Vegas Hotel', 'Bellagio Las Vegas',
       'Paris Las Vegas', 'The Westin las Vegas Hotel Casino & Spa']
count = 1
dict_hotels = {} #Empty dictionary to add values into

for i in list1:
    d[i] = count
    count+=1

In [261]:
d

{'Circus Circus Hotel & Casino Las Vegas': 1,
 'Excalibur Hotel & Casino': 2,
 'Monte Carlo Resort&Casino': 3,
 'Treasure Island- TI Hotel & Casino': 4,
 'Tropicana Las Vegas - A Double Tree by Hilton Hotel': 5,
 'Caesars Palace': 6,
 'The Cosmopolitan Las Vegas': 7,
 'The Palazzo Resort Hotel Casino': 8,
 'Wynn Las Vegas': 9,
 'Trump International Hotel Las Vegas': 10,
 'The Cromwell': 11,
 'Encore at wynn Las Vegas': 12,
 'Hilton Grand Vacations on the Boulevard': 13,
 "Marriott's Grand Chateau": 14,
 'Tuscany Las Vegas Suites & Casino': 15,
 'Hilton Grand Vacations at the Flamingo': 16,
 'Wyndham Grand Desert': 17,
 'The Venetian Las Vegas Hotel': 18,
 'Bellagio Las Vegas': 19,
 'Paris Las Vegas': 20,
 'The Westin las Vegas Hotel Casino & Spa': 21}

* There are 21 hotels in total.

### Visualization

In [212]:
# Make dataframe pipeline interactive
df = data.interactive(width=2200) # 

In [207]:
# # Select widget for Hotel name
# ho = data['Hotel_name'].unique()
# hotels = pn.widgets.Select(name='Hotels', options=['Circus Circus Hotel & Casino Las Vegas',
#        'Excalibur Hotel & Casino', 'Monte Carlo Resort&Casino',
#        'Treasure Island- TI Hotel & Casino',
#        'Tropicana Las Vegas - A Double Tree by Hilton Hotel',
#        'Caesars Palace', 'The Cosmopolitan Las Vegas',
#        'The Palazzo Resort Hotel Casino', 'Wynn Las Vegas',
#        'Trump International Hotel Las Vegas', 'The Cromwell',
#        'Encore at wynn Las Vegas',
#        'Hilton Grand Vacations on the Boulevard',
#        "Marriott's Grand Chateau", 'Tuscany Las Vegas Suites & Casino',
#        'Hilton Grand Vacations at the Flamingo', 'Wyndham Grand Desert',
#        'The Venetian Las Vegas Hotel', 'Bellagio Las Vegas',
#        'Paris Las Vegas', 'The Westin las Vegas Hotel Casino & Spa'])

# hotels

# # Radio buttons for hotels
# yaxis_stars = pn.widgets.RadioButtonGroup(
#     name='Y axis', 
#     options=['Score','Nr_hotel_reviews'],
#     button_type='success'
# )
# yaxis_stars
# # Need to connect data pipeline to widgets so that they are getting updated in real time.

# hotels_pipeline = (
#     df[
#         (df.Hotel_name == hotels)
#         #(df.Score)
#         #(df.User_country)
#     ]
#     .drop(['Hotel_name'], axis=1)
#     .groupby(['User_country','Review_month'])[yaxis_stars].mean()
#      .to_frame()
#      .reset_index()
#      .sort_values(by='User_country')  
#      .reset_index(drop=True)
#  )

# hotels_plot = hotels_pipeline.hvplot(x = 'Review_month', y=yaxis_stars, by='User_country',line_width=2, title="Score by customers based on Country")
# hotels_plot 

In [286]:
co2_source_bar_plot = hotels_pipeline.hvplot(kind='hist', 
                                                     x='Tennis_court', 
                                                     #y=["YES","NO"], 
                                                     title='Features')
co2_source_bar_plot

In [297]:
# Select widget for Hotel name and Score
ho = data['Hotel_stars'].unique()
hotel_stars = pn.widgets.Select(name='Hotel stars', options=[3,4,5])

# Radio buttons for hotels
yaxis_stars = pn.widgets.RadioButtonGroup(
    name='Y axis', 
    options=['Score','Nr_hotel_reviews'],
    button_type='success'
)
yaxis_stars


# Need to connect data pipeline to widgets so that they are getting updated in real time.

hotels_stars_pipeline = (
    df[
        (df.Hotel_stars == hotel_stars)
        #(df.Score)
        #(df.User_country)
    ]
    .groupby(['Hotel_name'])[yaxis_stars].mean()
     .to_frame()
     .reset_index()
     #.sort_values(by='User_country')  
     .reset_index(drop=True)
 )

hotels_starts_plot = hotels_stars_pipeline.hvplot(x = 'Hotel_name', y=yaxis_stars, line_width=2, title="Score of Hotels based on Hotel stars",width=500, height=300)
hotels_starts_plot.opts(fontsize=({'xticks': 8})) 

In [300]:
# Select widget for Hotel name and Score

ho_stars = data['Hotel_stars'].unique()
hotel_stars = pn.widgets.Select(name='Hotel stars', options=[3,4,5])

ho_names = data['Hotel_name'].unique()
hotels = pn.widgets.Select(name='Hotels', options=['Circus Circus Hotel & Casino Las Vegas',
       'Excalibur Hotel & Casino', 'Monte Carlo Resort&Casino',
       'Treasure Island- TI Hotel & Casino',
       'Tropicana Las Vegas - A Double Tree by Hilton Hotel',
       'Caesars Palace', 'The Cosmopolitan Las Vegas',
       'The Palazzo Resort Hotel Casino', 'Wynn Las Vegas',
       'Trump International Hotel Las Vegas', 'The Cromwell',
       'Encore at wynn Las Vegas',
       'Hilton Grand Vacations on the Boulevard',
       "Marriott's Grand Chateau", 'Tuscany Las Vegas Suites & Casino',
       'Hilton Grand Vacations at the Flamingo', 'Wyndham Grand Desert',
       'The Venetian Las Vegas Hotel', 'Bellagio Las Vegas',
       'Paris Las Vegas', 'The Westin las Vegas Hotel Casino & Spa'])

# Radio buttons for hotels
yaxis_stars = pn.widgets.RadioButtonGroup(
    name='Y axis', 
    options=['Score'],
    button_type='success'
)
yaxis_stars
# Need to connect data pipeline to widgets so that they are getting updated in real time.

hotels_pipeline = (
    df[
        (df.Hotel_stars==hotel_stars)&
        (df.Hotel_name == hotels)
        #(df.Score)
        #(df.User_country)
    ]
    .groupby(['Hotel_name','Hotel_stars','Nr_rooms','Tennis_court','Pool','Gym','Spa','Casino','Free_internet'])[yaxis_stars].mean()
     .to_frame()
     .reset_index()
     #.sort_values(by='User_country')  
     .reset_index(drop=True)
 )
hotels_pipeline

hotels_table = hotels_pipeline.pipe(pn.widgets.Tabulator, pagination='remote', page_size = 10, sizing_mode='stretch_width') 
hotels_table